# 03 — Model Definition (RTM)

## Part A (v0, pipeline-critical)
Compute a deterministic latent exposure proxy `E_hat` from water exposure priors and export:

- `outputs/rtm/water_exposure_Ehat_v0.parquet`
- `outputs/rtm/water_exposure_Ehat_v0_stats.json`

**This is NOT a hazard model and NOT flood probability.**

## Part B (v1+, optional)
Bayesian inference scaffold. **Disabled by default** and requires a real outcome / hazard intensity to condition on.


In [1]:
import os
from pathlib import Path
import json

import numpy as np
import pandas as pd


In [2]:
# Prefer setting HABNETIC_DATA in your shell, but allow a local default for convenience
os.environ.setdefault("HABNETIC_DATA", r"C:\Users\C.Price\Habnetic\data")

DATA_ROOT = os.environ.get("HABNETIC_DATA")
if not DATA_ROOT:
    raise KeyError(
        "HABNETIC_DATA not set.\n"
        "In PowerShell:\n"
        "$env:HABNETIC_DATA='C:\\Users\\C.Price\\Habnetic\\data'"
    )

DATA_ROOT = Path(DATA_ROOT)
DATA_ROOT


WindowsPath('C:/Users/C.Price/Habnetic/data')

In [3]:
priors_path = DATA_ROOT / "processed" / "RTM" / "priors" / "building_water_proximity.parquet"
print("Reading:", priors_path)

df = pd.read_parquet(priors_path)
print("Rows, cols:", df.shape)

# --- Ensure we have a stable building id column named 'bldg_id' ---
if "bldg_id" not in df.columns:
    # if an older file stored it as index, recover it
    if df.index.name == "bldg_id":
        df = df.reset_index()
    else:
        raise ValueError(
            "No 'bldg_id' column found (and index isn't named 'bldg_id').\n"
            "Fix upstream: normalize/clip must write bldg_id and exposure script must carry it."
        )

required = {
    "bldg_id",
    "dist_to_water_m",
    "water_len_density_250m",
    "water_len_density_500m",
    "water_len_density_1000m",
}
missing = required - set(df.columns)
if missing:
    raise ValueError(f"Missing required columns: {sorted(missing)}")

# Integrity: bldg_id must be unique + non-null (for joins)
if df["bldg_id"].isna().any():
    raise ValueError("bldg_id contains NaNs")
if df["bldg_id"].duplicated().any():
    dup_n = int(df["bldg_id"].duplicated().sum())
    raise ValueError(f"bldg_id is not unique ({dup_n} duplicates). Fix upstream.")

df.head()


Reading: C:\Users\C.Price\Habnetic\data\processed\RTM\priors\building_water_proximity.parquet
Rows, cols: (221324, 16)


,building,building:levels,building:height,height,building:material,roof:shape,roof:material,roof:height,roof:levels,name,addr:postcode,bldg_id,dist_to_water_m,water_len_density_250m,water_len_density_500m,water_len_density_1000m
0,yes,None,None,None,None,None,None,None,None,None,None,305012,72.742204,0.000963,0.000616,0.000455
1,service,None,None,None,None,None,None,None,None,None,None,313960,48.135264,0.002069,0.000838,0.000417
2,yes,None,None,None,None,None,None,None,None,None,None,313263,229.183788,0.001011,0.000767,0.000439
3,yes,None,None,None,None,None,None,None,None,None,None,310491,247.661546,0.000337,0.000673,0.000462
4,yes,None,None,None,None,None,None,None,None,None,None,313127,251.657334,0.000000,0.000749,0.000440


In [4]:
eps = 1e-9

# Distance: closer => higher exposure
# x_d = -log(d + 1.0)
df["x_d"] = -np.log(df["dist_to_water_m"].clip(lower=0) + 1.0)

# Water length density: x_r = log(rho_r + eps)
for r in [250, 500, 1000]:
    col = f"water_len_density_{r}m"
    df[f"x_{r}"] = np.log(df[col].clip(lower=0) + eps)

X_cols = ["x_d", "x_250", "x_500", "x_1000"]

# Standardize to z-scores
X = df[X_cols].to_numpy(dtype=float)
X_mean = X.mean(axis=0)
X_std  = X.std(axis=0)
X_std[X_std == 0] = 1.0

Z = (X - X_mean) / X_std
z_cols = ["z_d", "z_250", "z_500", "z_1000"]
df[z_cols] = Z

df[z_cols].describe()


,z_d,z_250,z_500,z_1000
count,2.213240e+05,2.213240e+05,2.213240e+05,2.213240e+05
mean,-4.023297e-16,3.421022e-16,3.980919e-16,-6.199319e-16
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-2.622094e+00,-1.268695e+00,-2.385862e+00,-5.290492e+00
25%,-6.321241e-01,-1.268695e+00,1.472925e-01,-7.040715e-02
50%,-4.412669e-02,6.282030e-01,3.932753e-01,2.429983e-01
75%,5.536877e-01,8.316384e-01,5.359612e-01,4.382702e-01
max,5.411108e+00,1.295231e+00,1.087652e+00,1.508299e+00


In [5]:
df["E_hat"] = df[z_cols].mean(axis=1)
df["E_hat"].describe()


count    2.213240e+05
mean    -6.388733e-17
std      8.402606e-01
min     -2.891786e+00
25%     -3.957054e-01
50%      3.127026e-01
75%      5.625253e-01
max      2.268184e+00
Name: E_hat, dtype: float64

In [6]:
out_dir = Path("outputs") / "rtm"
out_dir.mkdir(parents=True, exist_ok=True)

out_path = out_dir / "water_exposure_Ehat_v0.parquet"
stats_path = out_dir / "water_exposure_Ehat_v0_stats.json"

# Canonical join key is bldg_id
out = df[["bldg_id", "E_hat", *z_cols]].copy()

# Optional compatibility alias (uncomment if you really want a 'fid' column too)
# out["fid"] = out["bldg_id"]

out.to_parquet(out_path, index=False)

stats = {
    "schema_version": "0.1.0",
    "dataset": str(priors_path).replace("\\", "/"),
    "id_column": "bldg_id",
    "eps": float(eps),
    "features": X_cols,
    "transforms": {
        "x_d": "-log(dist_to_water_m + 1.0)",
        "x_250": "log(water_len_density_250m + eps)",
        "x_500": "log(water_len_density_500m + eps)",
        "x_1000": "log(water_len_density_1000m + eps)",
    },
    "standardization": {
        "mean": {k: float(v) for k, v in zip(X_cols, X_mean)},
        "std":  {k: float(v) for k, v in zip(X_cols, X_std)},
        "z_definition": "(x - mean) / std",
    },
    "E_hat_definition": "mean(z_d, z_250, z_500, z_1000)",
    "interpretation_constraints": [
        "E_hat is a deterministic exposure proxy, not a hazard model.",
        "No flood probability or risk is implied.",
        "Use for relative ranking / clustering until hazard intensity + outcomes exist."
    ],
}

stats_path.write_text(json.dumps(stats, indent=2), encoding="utf-8")

print("Wrote:", out_path)
print("Wrote:", stats_path)
out.head()


Wrote: outputs\rtm\water_exposure_Ehat_v0.parquet
Wrote: outputs\rtm\water_exposure_Ehat_v0_stats.json


,bldg_id,E_hat,z_d,z_250,z_500,z_1000
0,305012,0.536838,0.963020,0.802751,0.373586,0.007996
1,313960,0.677579,1.383126,0.917711,0.437259,-0.027780
2,313263,0.251841,-0.214838,0.809959,0.418931,-0.006689
3,310491,0.189019,-0.294736,0.644668,0.391794,0.014349
4,313127,-0.292821,-0.311231,-1.268695,0.414013,-0.005372


In [7]:
assert len(out) == len(df), "Row count changed unexpectedly"
assert out["bldg_id"].isna().sum() == 0, "bldg_id contains NaNs"
assert out["E_hat"].isna().sum() == 0, "E_hat contains NaNs"

# z-score sanity (means near 0, std near 1)
z_means = out[z_cols].mean()
z_stds = out[z_cols].std()
print("z means:\n", z_means)
print("\nz stds:\n", z_stds)

# Quick look at top exposure
out.sort_values("E_hat", ascending=False).head(10)


z means:
 z_d      -4.023297e-16
z_250     3.421022e-16
z_500     3.980919e-16
z_1000   -6.199319e-16
dtype: float64

z stds:
 z_d       1.000002
z_250     1.000002
z_500     1.000002
z_1000    1.000002
dtype: float64


,bldg_id,E_hat,z_d,z_250,z_500,z_1000
123707,735944,2.268184,5.313390,1.196664,1.067975,1.494705
123709,736490,2.095141,4.624520,1.196003,1.068601,1.491441
99312,126301,2.012785,5.403740,1.052544,0.756217,0.838637
123824,735858,2.007317,4.266858,1.205689,1.068791,1.487930
153981,1183111,1.998238,5.345733,1.130420,0.772886,0.743912
154014,1186929,1.994146,5.328986,1.125304,0.777944,0.744348
149768,1212048,1.977300,5.317460,1.079578,0.733277,0.778885
193740,925288,1.976405,5.362105,1.065248,0.772450,0.705816
153982,1184239,1.973173,5.243708,1.130416,0.775024,0.743545
153876,1207365,1.963251,5.249571,1.116283,0.797825,0.689324


## QGIS sanity check (required)

1) Load:
- `processed/RTM/derived/buildings_rtm.gpkg`

2) Join:
- Join `outputs/rtm/water_exposure_Ehat_v0.parquet` on `bldg_id`

3) Style:
- Graduated color on `E_hat`

Expectation:
- high `E_hat` clusters near dense canals/rivers/harbor network.

If this fails, fix the transforms/inputs now, not later.


# Part B — Bayesian layer (v1+ only)

This section is disabled unless you have a **real outcome / hazard intensity** to condition on.

If you fit against `observed=np.zeros(...)`, you will learn nothing and produce misleading outputs.

Leave `RUN_BAYES = False` until you have:
- a hazard intensity variable (e.g., flood depth proxy, rainfall extreme), and/or
- an outcome proxy (damage, downtime, cost), even if synthetic at first.


In [8]:
RUN_BAYES = False  # Set True only when y_obs exists


In [9]:
if RUN_BAYES:
    import pymc as pm
    import arviz as az

    # --- You must define y_obs before proceeding ---
    # Example placeholder:
    # y_obs = df["some_outcome_or_intensity"].to_numpy(dtype=float)

    raise NotImplementedError(
        "Define y_obs (outcome / hazard intensity) and a likelihood model before enabling RUN_BAYES."
    )
